In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from modeling.GRU_recommend_concat import *
from modeling.evaluation_trip import *
from sklearn.preprocessing import LabelEncoder

# Read Data and preprocess

In [2]:
# Read Session Data
data = pd.read_excel('../최종DB관리/Session_Detail_DB/Session_train_totalv2.xlsx')

In [3]:
ids=data[data["Spot"]=='구리코 네온'].Session_id
check = 0
for i in ids:
    sums=data[data["Session_id"]==i]["Spot"].isin(['도톤보리']).sum()
    if sums == 0:
        check += 1
        data.loc[(data["Session_id"]==i)&(data["Spot"]=="구리코 네온"),"Spot"]="도톤보리"
    else:
        data.loc[(data["Session_id"]==i)&(data["Spot"]=="구리코 네온"),"Spot"]="삭제"

In [4]:
data2=data[data["Spot"]!='삭제']

In [5]:
# Read item data
spot = pd.read_excel('../최종DB관리/Spot_VSS/2019.02.13.최종_통합DB2.xlsx')

In [6]:
spot["city_sub"]=spot.city +'_' + spot.sublocality
lab = LabelEncoder()
spot["city_sub_encoder"] = lab.fit_transform(spot["city_sub"])

In [7]:
# merge item and session (with attribute that will use in model)
data_spot = data.merge(spot[["ID2","Spot","Tag_encoder",'City_encoder','city_sub_encoder',"State_encoder"]],on="Spot",how="left")

In [8]:
# session_length 1 delete
lst_1=list(data_spot.groupby("Session_id").size().index[np.where(data_spot.groupby('Session_id').size()==1)[0]])
data_spot = data_spot[~data_spot.Session_id.isin(lst_1)]

In [9]:
# train test split
train = data_spot[:21003]
test = data_spot[21003:]

# Train model

In [10]:
class args():
    is_training = True
    retraining = False
    rnn_size = [4000,150] #  ,1000, 20, 5 
    n_epochs = 50
    batch_size = 50
    dropout_p_hidden= 0.5
    learning_rate = 0.0001
    decay = 0.96
    decay_steps = 5e3
    initializer = 'xavier'
    init_as_normal = False
    reset_after_session = True
    session_key = 'Session_id'
    attribute_key =["ID2",'City_encoder'] # input attribute
    item_key = 'Spot'
    time_key = 'Session_num'
    grad_cap = 0
    attribute = [len(spot),spot.City_encoder.nunique()] #, ,spot.Tag_encoder.nunique()  ,spot.city_sub_encoder.nunique()
    n_items = len(spot)
    max_ckpt_keep = 100
    test_model = 1
    sigma=0
    checkpoint_dir = 'ckpt_model'
    loss = 'top1'
    final_act = 'linear'
    hidden_act = 'tanh'
    item_table = spot

In [11]:
tf.reset_default_graph()

In [12]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
arg = args()
models = GRU4Rec(sess,arg)

In [13]:
models.fit(train)

fitting model....
Epoch 0	Step 1	lr: 0.000100	loss: 0.990551
Epoch: 0 cost: 0.9176268577575684
Epoch: 1 cost: 0.8406726717948914
Epoch: 2 cost: 0.810366153717041
Epoch: 3 cost: 0.7922488451004028
Epoch: 4 cost: 0.776819109916687
Epoch: 5 cost: 0.7658964395523071
Epoch: 6 cost: 0.754551887512207
Epoch: 7 cost: 0.7457354664802551
Epoch: 8 cost: 0.737910270690918
Epoch: 9 cost: 0.7315874099731445
Epoch: 10 cost: 0.7234575748443604


KeyboardInterrupt: 

# Test model

In [14]:
class args():
    is_training = False
    retraining = False
    rnn_size = [4000,150] #  ,1000, 20, 5 
    n_epochs = 50
    batch_size = 50
    dropout_p_hidden= 1.0
    learning_rate = 0.00001
    decay = 0.96
    decay_steps = 5e3
    initializer = 'xavier'
    init_as_normal = False
    reset_after_session = True
    session_key = 'Session_id'
    attribute_key =["ID2",'City_encoder'] # input attribute
    item_key = 'Spot'
    time_key = 'Session_num'
    grad_cap = 0
    attribute = [len(spot),spot.City_encoder.nunique()] #, ,spot.Tag_encoder.nunique()  ,spot.city_sub_encoder.nunique()
    n_items = len(spot)
    max_ckpt_keep = 100
    test_model = 10
    sigma=0
    checkpoint_dir = 'ckpt_model'
    loss = 'top1'
    final_act = 'linear'
    hidden_act = 'tanh'
    item_table = spot

In [15]:
tf.reset_default_graph()

In [16]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
arg = args()
models = GRU4Rec(sess,arg)

INFO:tensorflow:Restoring parameters from ckpt_model/attribute_model-10


In [15]:
# Check epoch 10 each performance
for i in range(0,10):
    tf.reset_default_graph()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    arg.test_model = i
    models = GRU4Rec(sess,arg)
    l_old=evaluate_sessions_batch(models,test, session_key= 'Session_id', item_key='Spot', time_key= 'Session_num',batch_size=50)
    print("Epoch:",i)
    print("Recall :",l_old[0])
    print("mrr :",l_old[1])
    print("Unique_recommend_lst",len(set(np.array(l_old[2].Rec_lst.tolist()).reshape(-1))))
    print('-----------------------------------------------------------------------------')

INFO:tensorflow:Restoring parameters from ckpt_model/attribute_model-0


D:\2019.01.04~여행앱개발\TripRecommendation\modeling\evaluation_trip.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_data.sort_values([session_key,time_key],inplace=True)


Recall [0.12073074 0.30436855 0.40127085 0.450834   0.47466243]
mrr [0.12073074 0.18859412 0.20195841 0.20584771 0.20718354]
Unique_recommend_lst 645
INFO:tensorflow:Restoring parameters from ckpt_model/attribute_model-1
Recall [0.13582208 0.33264496 0.4141382  0.45528197 0.48339952]
mrr [0.13582208 0.20592004 0.21698993 0.22024224 0.22183729]
Unique_recommend_lst 569
INFO:tensorflow:Restoring parameters from ckpt_model/attribute_model-2
Recall [0.13518666 0.34217633 0.42366958 0.46528991 0.49626688]
mrr [0.13518666 0.20851734 0.21941305 0.22271921 0.22446935]
Unique_recommend_lst 582
INFO:tensorflow:Restoring parameters from ckpt_model/attribute_model-3
Recall [0.13550437 0.34344718 0.42922955 0.47275616 0.50738681]
mrr [0.13550437 0.21006619 0.22165891 0.22516615 0.22710172]
Unique_recommend_lst 578
INFO:tensorflow:Restoring parameters from ckpt_model/attribute_model-4
Recall [0.13613979 0.35266084 0.4355838  0.48292295 0.51850675]
mrr [0.13613979 0.21380725 0.22499981 0.22875928 0.2

# Retraining

In [17]:
class args():
    is_training = True
    retraining = True
    rnn_size = [4000,150] #  ,1000, 20, 5 
    n_epochs = 50
    batch_size = 50
    dropout_p_hidden= 0.5
    learning_rate = 0.00001
    decay = 0.96
    decay_steps = 5e3
    initializer = 'xavier'
    init_as_normal = False
    reset_after_session = True
    session_key = 'Session_id'
    attribute_key =["ID2",'City_encoder'] # input attribute
    item_key = 'Spot'
    time_key = 'Session_num'
    grad_cap = 0
    attribute = [len(spot),spot.City_encoder.nunique()] #, ,spot.Tag_encoder.nunique()  ,spot.city_sub_encoder.nunique()
    n_items = len(spot)
    max_ckpt_keep = 100
    test_model = 10
    sigma=0
    checkpoint_dir = 'ckpt_model'
    loss = 'top1'
    final_act = 'linear'
    hidden_act = 'tanh'
    item_table = spot

In [21]:
tf.reset_default_graph()

In [22]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
arg = args()
models = GRU4Rec(sess,arg)

INFO:tensorflow:Restoring parameters from ckpt_model/attribute_model-10


In [23]:
models.fit(train)

fitting model....


KeyboardInterrupt: 